# Extract actual optimisation parameters to a table

Plot solution vectors from multiple MFILEs as a LateX table.

In [13]:
# Reload Process each time (keep editable install up-to-date)
%load_ext autoreload
%autoreload 2

import process.io.plot_solutions as ps
from pathlib import Path
import pandas as pd
from typing import Sequence

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
def opt_param_table(runs_metadata: Sequence[ps.RunMetadata]) -> pd.DataFrame:
    """Create an easy to read table of optimisation parameters for different runs.

    :param runs_metadata: metadata for mfiles to tabulate
    :type runs_metadata: Sequence[ps.RunMetadata]
    :return: dataframe in required form to plot
    :rtype: pd.DataFrame
    """
    # Create dataframe from runs metadata: mfile data with a tag for each run
    results_df = ps._create_df_from_run_metadata(runs_metadata)

    # Filter for tag, actual values of optimisation parameters and objective function
    opt_params_df = ps._filter_vars_of_interest(
        results_df, opt_param_value_pattern=ps.OPT_PARAM_VALUE_REGEX
    )

    # Now separate optimisation parameter values from their names
    opt_params_values_df = opt_params_df.filter(
        regex=f"{ps.OPT_PARAM_VALUE_REGEX}|{ps.TAG}"
    )
    opt_params_names_df = opt_params_df.filter(regex=ps.NORM_OPT_PARAM_NAME_REGEX)

    # Replace xcm--- optimisation parameter column headers with actual var names
    # in sub-df: allows plotting showing actual var names
    # eg. column headers "tag, xcm001, xcm002, ..." become
    # "tag, bt, rmajor, ..."
    # Normalising row may have been filtered out; reset index to ensure
    # opt param names in row 0
    opt_params_names_df_reset = opt_params_names_df.reset_index(drop=True)
    opt_params_names = opt_params_names_df_reset.loc[0].values.tolist()

    # Need to include tag column as first column header
    opt_params_names_with_tag = opt_params_names.copy()
    opt_params_names_with_tag.insert(0, ps.TAG)
    opt_params_values_with_names_df = opt_params_values_df.set_axis(
        opt_params_names_with_tag, axis=1
    )

    # This makes a wide df: difficult to present
    # Melt to simplify
    melted = opt_params_values_with_names_df.melt(id_vars="tag")
    # Pivot to get desired form
    pivoted = melted.pivot(index="variable", columns="tag", values="value")
    # Remove index name
    pivoted = pivoted.reset_index()
    return pivoted

In [15]:
data_dir = Path.cwd()
runs_metadata = [
    ps.RunMetadata(data_dir / "lt_MFILE.DAT", "min rmajor"),
    ps.RunMetadata(data_dir / "lt_max_net_elec_MFILE.DAT", "max net elec"),
]

opt_param_table_df = opt_param_table(
    runs_metadata=runs_metadata,
)

opt_param_table_df = opt_param_table_df.rename(
    columns={
        "variable": "Parameter",
        "max net elec": "max $P_{elec}^{net}$",
        "min rmajor": "min $R_{major}$",
    }
)

# Add in latex and description cols
opt_param_desc = {
    "beta": [r"$\beta$", "Plasma beta"],
    "bore": ["$R_{CS}$", "Central solenoid inboard radius (m)"],
    "bt": ["$B_T$", "Toroidal field on axis (T)"],
    "coheof": [
        "$J_{CS}$",
        "Central solenoid overall current density at end of flat-top (A/m2)",
    ],
    "cpttf": ["$I_{TF, turn}$", "TF coil current per turn (A)"],
    "dene": ["$n_e$", "Electron density (/m3)"],
    "dr_tf_wp": ["$\Delta R_{WP}$", "Radial thickness of winding pack (m)"],
    "fcohbop": [
        "$f_{J_{CS}}$",
        "Ratio of central solenoid overall current density at beginning of pulse / end of flat-top",
    ],
    "fcutfsu": ["$f_{TF_{Cu}}$", "Copper fraction of cable conductor (TF coils)"],
    "fimp(13)": ["$f_{Xe}$", "Xenon density fraction relative to electron density"],
    "fvsbrnni": [
        "$f_{I_p, NI}$",
        "Fraction of the plasma current produced by non-inductive means",
    ],
    "hfact": ["$H$", "H factor on energy confinement times, radiation corrected"],
    "oh_steel_frac": ["$f_{CS_{steel}}$", "Central solenoid steel fraction"],
    "ohcth": ["$\Delta R_{CS}$", "Central solenoid thickness (m)"],
    "q": ["$q^{95}$", "Safety factor 'near' plasma edge, equal to q95"],
    "ralpne": [r"$f_{n_{\alpha}, n_e}$", "Thermal alpha density/electron density"],
    "rmajor": ["$R_{major}", "Plasma major radius (m)"],
    "tdmptf": [
        "$t_{TF, dis}$",
        "Fast discharge time for TF coil in event of quench (s)",
    ],
    "te": ["$T_e$", "Volume averaged electron temperature (keV)"],
    "tfcth": ["$\Delta R_{TF, in}$", "Inboard TF coil thickness (m)"],
    "thkcas": [
        "$\Delta R_{TF case, in}$",
        "Inboard TF coil case outer (non-plasma side) thickness (m)",
    ],
    "thwcndut": ["$\Delta R_{TF cond}$", "TF coil conduit case thickness (m)"],
}

# Add latex and descriptions for each param
desc_dict = {"Symbol": [], "Description": []}
for param in opt_param_table_df["Parameter"].values:
    desc_dict["Symbol"].append(opt_param_desc[param][0])
    desc_dict["Description"].append(opt_param_desc[param][1])

desc_df = pd.DataFrame(desc_dict)
opt_param_table_df = pd.concat((opt_param_table_df, desc_df), axis="columns")

# Re-arrange col order
cols = opt_param_table_df.columns.to_list()
cols = [cols[0]] + [cols[3]] + [cols[2]] + [cols[1]] + [cols[4]]
opt_param_table_df = opt_param_table_df[cols]

# Make latex table
opt_param_table_df.to_latex(
    "opt_params_table.tex",
    float_format="%.2e",
    index=False,
    longtable=True,
    column_format="llrrp{0.45\linewidth}",
)
opt_param_table_df

,Parameter,Symbol,min $R_{major}$,max $P_{elec}^{net}$,Description
0,beta,$\beta$,3.290791e-02,3.342605e-02,Plasma beta
1,bore,$R_{CS}$,1.968259e+00,2.044679e+00,Central solenoid inboard radius (m)
2,bt,$B_T$,5.338571e+00,6.093892e+00,Toroidal field on axis (T)
3,coheof,$J_{CS}$,2.099150e+07,1.636166e+07,Central solenoid overall current density at en...
4,cpttf,"$I_{TF, turn}$",8.203100e+04,7.066068e+04,TF coil current per turn (A)
5,dene,$n_e$,7.975399e+19,8.168222e+19,Electron density (/m3)
6,dr_tf_wp,$\Delta R_{WP}$,5.323532e-01,7.537202e-01,Radial thickness of winding pack (m)
7,fcohbop,$f_{J_{CS}}$,9.324863e-01,9.317779e-01,Ratio of central solenoid overall current dens...
8,fcutfsu,$f_{TF_{Cu}}$,8.255325e-01,8.536375e-01,Copper fraction of cable conductor (TF coils)
9,fimp(13),$f_{Xe}$,5.592063e-04,8.573539e-04,Xenon density fraction relative to electron de...
